In [ ]:
#1) Detect when vehicle is being used  
#2) Match driver's phone and vehicle
#3) Track driving behavior
#4) Track if texting is enabled
#5) Run scoring model, on risky and distracted events
#6) Make data available for Scoring API

In [30]:
import json
import pandas as pd
from dateutil.parser import parse
from datetime import datetime

#Helper Functions
def parse_time(s):
    try:
        ret = parse(s)
    except ValueError:
        ret = datetime.utcfromtimestamp(s)
    return ret

#print(parse_time(data['lastTimestamp'][0]))

2018-06-02 22:04:18+00:00


In [1]:

#TelemetryByVinController #Header Parameters #x-id-token

#Grab vin, lastTimestamp, location-coordinates, telemetry-speed-value
with open('telemetry_api_response.txt') as f: 
    json_data = json.load(f)
json_data

{'vin': 'VINVINVINVINVIN01',
 'correlationId': 'correlation-id',
 'lastTimestamp': '2018-06-02T22:04:18Z',
 'location': {'type': 'Point', 'coordinates': [-88.3185833333333, 42.228175]},
 'vehicleName': 'CAMRY',
 'telemetry': {'fuelLevel': {'value': 26.46},
  'odometer': {'value': 5176, 'unit': 'mi'},
  'sunRoof': {'value': 2},
  'spareTirePressure': {'value': 127, 'unit': 'psi'},
  'flTirePressure': {'value': 36, 'unit': 'psi'},
  'driverWindow': {'value': 2},
  'rrWindow': {'value': 2},
  'speed': {'value': 0, 'unit': 'km/h'},
  'frTirePressure': {'value': 37, 'unit': 'psi'},
  'rlTirePressure': {'value': 37, 'unit': 'psi'},
  'passengerWindow': {'value': 2},
  'rrTirePressure': {'value': 37, 'unit': 'psi'},
  'rlWindow': {'value': 2}}}

In [2]:
d = json_data
df = pd.DataFrame.from_dict(d, orient='index').astype(str).sort_index()

df.T #columnn for each timestamp?
data = df.T

In [3]:
df_repeated = pd.concat([data]*3, ignore_index=True)
df_repeated

,correlationId,lastTimestamp,location,telemetry,vehicleName,vin
0,correlation-id,2018-06-02T22:04:18Z,"{'type': 'Point', 'coordinates': [-88.31858333...","{'fuelLevel': {'value': 26.46}, 'odometer': {'...",CAMRY,VINVINVINVINVIN01
1,correlation-id,2018-06-02T22:04:18Z,"{'type': 'Point', 'coordinates': [-88.31858333...","{'fuelLevel': {'value': 26.46}, 'odometer': {'...",CAMRY,VINVINVINVINVIN01
2,correlation-id,2018-06-02T22:04:18Z,"{'type': 'Point', 'coordinates': [-88.31858333...","{'fuelLevel': {'value': 26.46}, 'odometer': {'...",CAMRY,VINVINVINVINVIN01


In [4]:
#len(df_repeated.telemetry.values) #convert to array, #length is number of rows. 

In [5]:
odo = []

for i in df_repeated.telemetry.values:
    s = i
    json_acceptable_string = s.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    odo.append(d['odometer'].get("value"))
    
df_repeated['odometer'] = odo
#df_repeated

In [6]:
sp = []
for i in df_repeated.telemetry.values:
    s = i
    json_acceptable_string = s.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    sp.append(d['speed'].get("value"))
    
df_repeated['speed'] = sp
#df_repeated

In [7]:
gps_coord = []

for i in df_repeated.location.values:
    s = i
    json_acceptable_string = s.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    gps_coord.append(d['coordinates'])

In [8]:
df_repeated['gps_coord'] = gps_coord
#df_repeated

In [9]:
#df_repeated.columns
data = df_repeated[['vin','lastTimestamp', 'speed','gps_coord', 'odometer']]
data

,vin,lastTimestamp,speed,gps_coord,odometer
0,VINVINVINVINVIN01,2018-06-02T22:04:18Z,0,"[-88.3185833333333, 42.228175]",5176
1,VINVINVINVINVIN01,2018-06-02T22:04:18Z,0,"[-88.3185833333333, 42.228175]",5176
2,VINVINVINVINVIN01,2018-06-02T22:04:18Z,0,"[-88.3185833333333, 42.228175]",5176


In [17]:
#speed threshold
#gps_coord - geofencing, match with phone
#odo - distance traveled

In [32]:
data['lastTimestamp'] = data['lastTimestamp'].apply(lambda x: parse_time(x))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
data

,vin,lastTimestamp,speed,gps_coord,odometer
0,VINVINVINVINVIN01,2018-06-02 22:04:18+00:00,0,"[-88.3185833333333, 42.228175]",5176
1,VINVINVINVINVIN01,2018-06-02 22:04:18+00:00,0,"[-88.3185833333333, 42.228175]",5176
2,VINVINVINVINVIN01,2018-06-02 22:04:18+00:00,0,"[-88.3185833333333, 42.228175]",5176
